# Scraping Latest ICE Removals

https://trac.syr.edu/phptools/immigration/remove/

In [63]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import os

# webdriver_path =  "/Users/joaburkh/Downloads/geckodriver"
# webdriver_path = '/Users/joaburkh/Downloads/chromedriver-mac-x64/chromedriver'

ops = Options()
# ops.add_argument('--headless')
browser = webdriver.Firefox(options=ops)
# browser.headless = True
browser.get('https://trac.syr.edu/phptools/immigration/remove/')
browser.implicitly_wait(15)

In [64]:
# Set third axis (I don't know why the setting has to be done in reverse order)
axis3_name = 'Citizenship'
dropdown = browser.find_element(By.CSS_SELECTOR, "#col3head2 > #dimension_pick_col1")
dropdown.find_element(By.XPATH, f"//option[. = '{axis3_name}']").click()
browser.find_element(By.CSS_SELECTOR, "#col3head2 option:nth-child(14)").click()
  

In [65]:
# Set second axis
axis2_name = 'Status at Latest Entry into U.S.'
dropdown = browser.find_element(By.CSS_SELECTOR, "#col2head2 > #dimension_pick_col1")
dropdown.find_element(By.XPATH, f"//option[. = '{axis2_name}']").click()
browser.find_element(By.CSS_SELECTOR, "#col2head2 option:nth-child(16)").click()

In [66]:
# Set first axis
axis1_name = 'Fiscal Year Deported'
browser.find_element(By.ID, "dimension_pick_col1").click()
dropdown = browser.find_element(By.ID, "dimension_pick_col1")
dropdown.find_element(By.XPATH, f"//option[. = '{axis1_name}']").click()

In [67]:
# Get list of tables
tables = browser.find_elements(By.CLASS_NAME, 'Table')

In [68]:
browser.find_element(By.LINK_TEXT, str(2013)).click()

In [74]:
import time
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Get list of entries along first axis (to be clicked on one by one)
table1_rows = [row.replace(',', '') for row in tables[0].text.split('\n')[2:]]

# Initialize data dict
data = {}

# Iterate over each year
for t1_row_text in [r for r in table1_rows if int(t1_row_text.rsplit(' ', 1)[0]) not in (2012, 2009)]:
    # Calculate year and initialize data dictionary entry
    year = int(t1_row_text.rsplit(' ', 1)[0])
    data[year] = {}

    # Click on the row
    t1_row_to_click = browser.find_element(By.LINK_TEXT, str(year))
    # try:
    #     t1_row_to_click = WebDriverWait(browser, 20).until(
    #         EC.presence_of_element_located((By.LINK_TEXT, str(year)))
    #     )
    # finally:
    #     print(f'{year} could not be found. Quitting...')
    #     browser.close()

    t1_row_to_click.click()

    # Recalculate Tables
    time.sleep(0.1)
    tables = browser.find_elements(By.CLASS_NAME, 'Table')

    # Get list of entries along second axis (to be clicked on one by one)
    table2_rows = [row.rsplit(' ', 1) for row in tables[1].text.split('\n')[2:]]
    table2_rows = [[row[0], row[1].replace(',', '')] for row in table2_rows]
    
    # Iterate over each nationality for current year
    for t2_row_text in table2_rows:
        # Calculate Nationality
        nationality = t2_row_text[0]

        # Click on the row
        # try:
        #     t2_row_to_click = WebDriverWait(browser, 20).until(
        #     EC.presence_of_element_located((By.LINK_TEXT, t2_row_text[0]))
        # )
        # finally:
        #     print(f'{t2_row_text[0]} could not be found. Quitting...')
        #     browser.close()
        t2_row_to_click = browser.find_element(By.LINK_TEXT, nationality)
        t2_row_to_click.click()

        # Recalculate Tables
        time.sleep(0.1)
        tables = browser.find_elements(By.CLASS_NAME, 'Table')

        # Get a list of entries along third axis (to be copied as-is)
        table3_rows = [row.replace(',', '').rsplit(' ', 1) for row in tables[2].text.split('\n')[2:]]
        table3_text = {row[0]: int(row[1]) for row in table3_rows}

        # Add to data dictionary
        data[year][nationality] = table3_text

In [77]:
# Export to HDF
import pandas as pd 
from pathlib import Path

data_df = pd.concat({k: pd.DataFrame(v).T for k, v in data.items()}, axis=0) # https://stackoverflow.com/a/54300940
data_df.to_hdf(Path.cwd() / 'TRAC_ICERemovals_Raw.hdf', key='ICERemovals_2003to2020')

C:\Users\Joseph\AppData\Local\Temp\ipykernel_27064\2306662016.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Pre-processing Data

In [33]:
import pandas as pd

# Import Data
data_df = pd.read_hdf('TRAC_ICERemovals_Raw.hdf')

# Set index name
data_df.index.rename(names=['Year', 'Nationality'], level=[0, 1], inplace=True)
data_df

Present Without Admission    Other  \
Year Nationality                                                    
2012 Mexico                                     265828.0  10277.0   
     Guatemala                                   38591.0   1088.0   
     Honduras                                    31217.0    387.0   
     El Salvador                                 18292.0    641.0   
     Dominican Republic                           1534.0    283.0   
...                                                  ...      ...   
2020 Slovenia                                        NaN      NaN   
     Malawi                                          NaN      NaN   
     Cayman Islands                                  NaN      NaN   
     Papua New Guinea                                NaN      NaN   
     Central African Republic                        NaN      NaN   

                               Non-Immigrant  Legal Permanent Resident  \
Year Nationality                                                         
2012 Mexico                           4776.0                    2897.0   
     Guatemala                         328.0                      85.0   
     Honduras                          282.0                      91.0   
     El Salvador                       144.0                     220.0   
     Dominican Republic                247.0                     603.0   
...                                      ...                       ...   
2020 Slovenia                            NaN                       NaN   
     Malawi                              NaN                       NaN   
     Cayman Islands                      NaN                       NaN   
     Papua New Guinea                    1.0                       NaN   
     Central African Republic            1.0                       NaN   

                               Unknown  Parolee  Immigrant  Visitor  Asylum  \
Year Nationality                                                              
2012 Mexico                     2003.0   1311.0      682.0    471.0   149.0   
     Guatemala                   352.0     19.0       45.0     26.0    76.0   
     Honduras                    341.0     57.0       38.0     23.0    15.0   
     El Salvador                 289.0     50.0       43.0     30.0    40.0   
     Dominican Republic          224.0     95.0      165.0     45.0     NaN   
...                                ...      ...        ...      ...     ...   
2020 Slovenia                      NaN      NaN        NaN      NaN     NaN   
     Malawi                        NaN      NaN        NaN      1.0     NaN   
     Cayman Islands                NaN      NaN        NaN      NaN     NaN   
     Papua New Guinea              NaN      NaN        NaN      NaN     NaN   
     Central African Republic      NaN      NaN        NaN      NaN     NaN   

                               Visa Waiver Program  Not Applicable  
Year Nationality                                                    
2012 Mexico                                    1.0             NaN  
     Guatemala                                 NaN             NaN  
     Honduras                                  1.0             NaN  
     El Salvador                               NaN             NaN  
     Dominican Republic                       11.0             NaN  
...                                            ...             ...  
2020 Slovenia                                  1.0             NaN  
     Malawi                                    NaN             NaN  
     Cayman Islands                            NaN             1.0  
     Papua New Guinea                          NaN             NaN  
     Central African Republic                  NaN             NaN  

[3370 rows x 11 columns]

Make sure that every year has same country rows and every country has the same decision columns

In [34]:
# Rectify missing country rows
unique_index1 = data_df.index.unique(0)
unique_index2 = data_df.index.unique(1)

new_index = pd.MultiIndex.from_product([unique_index1, unique_index2])

data_df = data_df.reindex(new_index, axis='index')


with pd.option_context('display.max_rows', 250):
    display(data_df.loc[(2015)])

,Present Without Admission,Other,Non-Immigrant,Legal Permanent Resident,Unknown,Parolee,Immigrant,Visitor,Asylum,Visa Waiver Program,Not Applicable
Nationality,,,,,,,,,,,
Mexico,115531.0,2167.0,2465.0,1377.0,659.0,341.0,360.0,168.0,2754.0,NaN,20583.0
Guatemala,30827.0,279.0,305.0,47.0,92.0,13.0,29.0,12.0,109.0,1.0,1611.0
Honduras,18283.0,109.0,334.0,46.0,100.0,15.0,27.0,6.0,124.0,NaN,1295.0
El Salvador,20299.0,248.0,122.0,126.0,88.0,11.0,33.0,8.0,88.0,NaN,779.0
Dominican Republic,936.0,95.0,133.0,368.0,62.0,36.0,103.0,20.0,2.0,5.0,194.0
Brazil,579.0,18.0,58.0,1.0,8.0,2.0,3.0,16.0,3.0,2.0,43.0
Ecuador,1098.0,36.0,49.0,28.0,19.0,8.0,9.0,5.0,4.0,1.0,56.0
Colombia,334.0,80.0,245.0,52.0,19.0,130.0,16.0,20.0,18.0,NaN,228.0
Nicaragua,669.0,20.0,63.0,27.0,7.0,2.0,6.0,2.0,14.0,NaN,50.0


In [35]:
# Change all NaN values to 0
data_df = data_df.fillna(value=0.0)

# Re-order DF by year (ascending) and country (ascending)
data_df = data_df.sort_index()
with pd.option_context('display.max_rows', 250):
    display(data_df)

Present Without Admission  Other  Non-Immigrant  \
Year Nationality                                                       
2003 Afghanistan                          14.0    7.0            0.0   
     Albania                              43.0   79.0           16.0   
     Algeria                              12.0   19.0            1.0   
     American Samoa                        0.0    0.0            0.0   
     Andorra                               0.0    0.0            0.0   
...                                        ...    ...            ...   
2020 Western Sahara                        0.0    0.0            0.0   
     Yemen                                 8.0    1.0            1.0   
     Yugoslavia                            0.0    2.0            0.0   
     Zambia                                0.0    1.0            2.0   
     Zimbabwe                              0.0    3.0            2.0   

                     Legal Permanent Resident  Unknown  Parolee  Immigrant  \
Year Nationality                                                             
2003 Afghanistan                          0.0     54.0      0.0        0.0   
     Albania                              1.0     80.0      0.0        1.0   
     Algeria                              0.0     35.0      0.0        0.0   
     American Samoa                       0.0      0.0      0.0        0.0   
     Andorra                              0.0      0.0      0.0        0.0   
...                                       ...      ...      ...        ...   
2020 Western Sahara                       0.0      0.0      0.0        0.0   
     Yemen                                0.0      1.0      0.0        0.0   
     Yugoslavia                           0.0      0.0      0.0        0.0   
     Zambia                               0.0      0.0      0.0        0.0   
     Zimbabwe                             0.0      0.0      0.0        0.0   

                     Visitor  Asylum  Visa Waiver Program  Not Applicable  
Year Nationality                                                           
2003 Afghanistan         2.0     0.0                  0.0             0.0  
     Albania             0.0     1.0                  0.0             0.0  
     Algeria             2.0     0.0                  0.0             0.0  
     American Samoa      0.0     0.0                  0.0             0.0  
     Andorra             0.0     0.0                  0.0             0.0  
...                      ...     ...                  ...             ...  
2020 Western Sahara      0.0     0.0                  0.0             0.0  
     Yemen               0.0     1.0                  0.0             1.0  
     Yugoslavia          1.0     0.0                  0.0             0.0  
     Zambia              0.0     0.0                  0.0             3.0  
     Zimbabwe            0.0     0.0                  0.0             6.0  

[3924 rows x 11 columns]

Merge Duplicate Country Names (identified manually)

In [36]:
# Remaining duplicates
dups = [
    ["Byelorussia (Belarus)", "Belarus"],
    ["Holland", "Netherlands"],
    ["Be Removed Fr...United States", "Be Removed From the United States"],
    ["British India...ean Territory", "British Indian Ocean Territory"],
    ["Federated Sta...of Micronesia", "Federated States of Micronesia"],
    ["St. Vincent a...he Grenadines", "St. Vincent and the Grenadines"],
    ["Stateless - A...ame A Country", "Stateless - Alien Unable To Name A Country"],
    ["The Republic ...shall Islands", "The Republic of the Marshall Islands"],
    ["Antigua-Barbuda",	"Antigua and Barbuda"],
    ["Antigua-Barbuda", "Antigua and Barbuda"],
    ["Burma", "Burma (Myanmar)"],
    ["China, Peoples Republic Of", "China"],
    ["Comoros", "Comoro Islands"],
    ["Dem Rep Of The Congo", "Democratic Republic of Congo"],
    ["Guinea Bissau", "Guinea-Bissau"],
    ["Ivory Coast", "Ivory Coast (Cote D'ivoire)"],
    ["Kyrgyzstan", "Kirghizia (Kyrgyzstan)"],
    ["Marshall Islands", "The Republic of the Marshall Islands"],
    ["Micronesia, Federated States Of", "Federated States of Micronesia"],
    ["Moldova", "Moldavia (Moldova)"],
    ["Macedonia", "North Macedonia"],
    ["Palau", "The Republic of Palau"],
    ["Sao Tome And Principe", "Sao Tome and Principe"],
    ["Slovakia", "Slovak Republic"],
    ["St. Vincent and the Grenadines", "St. Vincent-Grenadines"],
    ["Tajikistan", "Tajikistan (Tadzhik)"],
    ["Trinidad And Tobago", "Trinidad and Tobago"],
    ["Turks And Caicos Islands", "Turks and Caicos Islands"]
]
for year in data_df.index.unique(0):
    for dup, orig in dups:
        if (dup in data_df.loc[year].index.unique(0)):
            data_df.rename(index={dup: orig}, inplace=True)   
            data_df = data_df.groupby(level=[0,1]).sum()                              # Sum the now-identical rows

Assign Regional Names to Countries (identified manually)

In [37]:
filename_win = "C:\\Users\\Joseph\\UMich_MSE_PhD\\UMich_SI649_InformationVisualization\\Project_StaticViz\\possible_nationalities.csv"
filename_mac = '/Users/joaburkh/Library/CloudStorage/GoogleDrive-joaburkh@umich.edu/Other computers/My Laptop/UMich_MSE_PhD/UMich_SI649_InformationVisualization/Project_StaticViz/possible_nationalities.csv'

region_df = pd.read_csv(filename_win, usecols=[0,1,2])
region_df.set_index('Nationality', inplace=True)

data_df = data_df.join(region_df)

In [38]:
data_df.set_index('Region', append=True, inplace=True)
data_df.set_index('Subregion', append=True, inplace=True)
data_df = data_df.reorder_levels(['Year', 'Region', 'Subregion', 'Nationality'])
data_df.sort_index(inplace=True)

In [39]:
#Note: this step reveals any countries that haven't yet been assigned a region/subregion (look at NaN indices)
with pd.option_context('display.max_rows', 250):
    with pd.option_context('display.expand_frame_repr', True):
        with pd.option_context('display.width', 1000):
            display(data_df.loc[(2020)])

Present Without Admission  Other  Non-Immigrant  Legal Permanent Resident  Unknown  Parolee  Immigrant  Visitor  Asylum  Visa Waiver Program  Not Applicable
Region          Subregion              Nationality                                                                                                                                                                                       
Africa          Central Africa         Burundi                                                     0.0    0.0            0.0                       0.0      0.0      0.0        0.0      0.0     0.0                  0.0             1.0
                                       Central African Republic                                    0.0    0.0            1.0                       0.0      0.0      0.0        0.0      0.0     0.0                  0.0             0.0
                                       Congo                                                       1.0    1.0            0.0                       0.0      1.0      0.0        0.0      0.0     0.0                  0.0             2.0
                                       Democratic Republic of Congo                               11.0    2.0            2.0                       0.0      2.0      0.0        0.0      0.0    17.0                  0.0             7.0
                                       Rwanda                                                      1.0    1.0            1.0                       0.0      0.0      0.0        0.0      0.0     0.0                  0.0             1.0
                                       South Sudan                                                 1.0    0.0            0.0                       2.0      1.0      0.0        1.0      0.0     8.0                  0.0            12.0
                Eastern Africa         Comoro Islands                                              0.0    0.0            0.0                       0.0      0.0      0.0        0.0      0.0     0.0                  0.0             0.0
                                       Djibouti                                                    0.0    0.0            0.0                       0.0      0.0      0.0        0.0      0.0     0.0                  0.0             0.0
                                       Eritrea                                                     1.0    1.0            1.0                       0.0      5.0      0.0        0.0      0.0     5.0                  0.0             1.0
                                       Ethiopia                                                    5.0    7.0            3.0                       0.0      4.0      0.0        0.0      0.0     4.0                  0.0            10.0
                                       Kenya                                                       5.0    6.0           22.0                       3.0      2.0      0.0        0.0      1.0     2.0                  0.0            18.0
                                       Madagascar                                                  0.0    0.0            0.0                       0.0      0.0      0.0        0.0      0.0     0.0                  0.0             0.0
                                       Malawi                                                      0.0    0.0            0.0                       0.0      0.0      0.0        0.0      1.0     0.0                  0.0             0.0
                                       Mozambique                                                  0.0    0.0            1.0                       1.0      0.0      0.0        0.0      0.0     0.0                  0.0             0.0
                                       Seychelles                                                  0.0    0.0            1.0                       0.0      0.0      0.0        0.0      0.0     0.0                  0.0             0.0
                                       Somalia                                                     

Add a total column to make plotting easier

In [40]:
data_df['Total'] = data_df.sum(axis=1)

In [41]:
data_df

Present Without Admission  \
Year Region        Subregion              Nationality                                               
2003 Africa        Central Africa         Burundi                                             0.0   
                                          Central African Republic                            0.0   
                                          Congo                                               4.0   
                                          Democratic Republic of Congo                        1.0   
                                          Rwanda                                              1.0   
...                                                                                           ...   
2020 South America Northern South America Guyana                                              7.0   
                                          Suriname                                            0.0   
                                          Venezuela                                          18.0   
                   Western South America  Peru                                              142.0   
     Stateless     Stateless              Stateless                                           0.0   

                                                                        Other  \
Year Region        Subregion              Nationality                           
2003 Africa        Central Africa         Burundi                         2.0   
                                          Central African Republic        0.0   
                                          Congo                           8.0   
                                          Democratic Republic of Congo    6.0   
                                          Rwanda                          3.0   
...                                                                       ...   
2020 South America Northern South America Guyana                          4.0   
                                          Suriname                        0.0   
                                          Venezuela                       0.0   
                   Western South America  Peru                            5.0   
     Stateless     Stateless              Stateless                       0.0   

                                                                        Non-Immigrant  \
Year Region        Subregion              Nationality                                   
2003 Africa        Central Africa         Burundi                                 1.0   
                                          Central African Republic                0.0   
                                          Congo                                   4.0   
                                          Democratic Republic of Congo            0.0   
                                          Rwanda                                  2.0   
...                                                                               ...   
2020 South America Northern South America Guyana                                  9.0   
                                          Suriname                                0.0   
                                          Venezuela                              70.0   
                   Western South America  Peru                                   26.0   
     Stateless     Stateless              Stateless                               4.0   

                                                                        Legal Permanent Resident  \
Year Region        Subregion              Nationality                                              
2003 Africa        Central Africa         Burundi                                            0.0   
                                          Central African Republic                           0.0   
                                          Congo                                              0.0   
                                          Demo

Convert all floats to int (cannot have fractions of people)

In [42]:
float_cols = data_df.select_dtypes(include=['float64']) # This will select float columns only
for col in float_cols.columns.values:
    data_df[col] = data_df[col].astype('int64')

data_df

Present Without Admission  \
Year Region        Subregion              Nationality                                               
2003 Africa        Central Africa         Burundi                                               0   
                                          Central African Republic                              0   
                                          Congo                                                 4   
                                          Democratic Republic of Congo                          1   
                                          Rwanda                                                1   
...                                                                                           ...   
2020 South America Northern South America Guyana                                                7   
                                          Suriname                                              0   
                                          Venezuela                                            18   
                   Western South America  Peru                                                142   
     Stateless     Stateless              Stateless                                             0   

                                                                        Other  \
Year Region        Subregion              Nationality                           
2003 Africa        Central Africa         Burundi                           2   
                                          Central African Republic          0   
                                          Congo                             8   
                                          Democratic Republic of Congo      6   
                                          Rwanda                            3   
...                                                                       ...   
2020 South America Northern South America Guyana                            4   
                                          Suriname                          0   
                                          Venezuela                         0   
                   Western South America  Peru                              5   
     Stateless     Stateless              Stateless                         0   

                                                                        Non-Immigrant  \
Year Region        Subregion              Nationality                                   
2003 Africa        Central Africa         Burundi                                   1   
                                          Central African Republic                  0   
                                          Congo                                     4   
                                          Democratic Republic of Congo              0   
                                          Rwanda                                    2   
...                                                                               ...   
2020 South America Northern South America Guyana                                    9   
                                          Suriname                                  0   
                                          Venezuela                                70   
                   Western South America  Peru                                     26   
     Stateless     Stateless              Stateless                                 4   

                                                                        Legal Permanent Resident  \
Year Region        Subregion              Nationality                                              
2003 Africa        Central Africa         Burundi                                              0   
                                          Central African Republic                             0   
                                          Congo                                                0   
                                          Demo

Save pre-processed dataframe to disk

In [43]:
data_df.to_hdf('TRAC_ICERemovals_Cleaned.hdf', key='ICERemovals_2003to2020')